# Final Project

## Due Thursday, December 12th at 5:00 PM EDT, no extensions

### worked with Muyang Guo

## 1. Running a community benchmark (15 pts)

For those working in pairs, one version of this question will be graded.

You are asked to take all of the necessary steps to run a meaningful benchmark code on pace-ice.  First we should talk about what a meaningful benchmark is.

### A meaningful benchmark should:

### a. Help someone working in a non-HPC domain understand / predict how useful a particular machine is to solving their problem.

### b. Report a machine-independent measure of performance, to allow for fair comparison and portability.

### c. Have an algorithm-independent statement of what the problem is (i.e. phrased in terms of inputs and outputs), to avoid artificially constraining the implementations.

### d. Be as simple as possible, so that the results of the benchmark are explainable and reproducible.

With these criteria in mind, you are welcome to select any accepted community benchmark with an open-source implementation.

- The benchmark implementation must be *open*, so that we may see what exactly is being run.
- An "accepted community" benchmark should ideally have a website describing itself, publishd benchmark results, and (ideally) a peer-reviewed in-depth description.


### Here are some recommendations that you could choose from:

### [HPLinpack](http://www.netlib.org/benchmark/hpl/): Dense Linear Algebra

### [HPCG](http://hpcg-benchmark.org/): Iterative Sparse Linear Algebra

### [Graph500](https://graph500.org/): Data-Intensive Graph Algorithms

### [HPGMG](http://crd.lbl.gov/departments/computer-science/PAR/research/hpgmg/): Multilevel PDE Solvers

### [LAMMPS](https://lammps.sandia.gov/index.html) ([benchmarks](https://lammps.sandia.gov/bench.html)): Molecular Dynamics

### [TensorFlow](tensorflow.org) ([benchmarks](https://github.com/tensorflow/benchmarks)): Machine Learning

---

**1.1 (1 pts):** In a cell below, tell me which benchmark you are choosing.  Provide a link.  If the benchmark is actually a suite of benchmarks, tell me which one you would like to focus on.  If there are citations for the benchmark, give me those, too, please.  After that, give:

- As complete a description as possible of the *problem* being solved.  Include scaling parameters like problem size $N$, and any other "free" parameters that can change between different runs of the benchmark.

- As complete a description as possible of the *value* of the benchmark: what quantity is being reported?

Then, tell me which type of pace-ice node you intend to use to test the benchmark.

**1.2 (4 pts):** In your own words, give me your assessment of the quality of the benchmark according to the four points (a), (b), (c), and (d) above.

- a. Describe some applications where the benchmark problem is relevant.  Benchmarks must walk a fine line between being to specific to one application but very predictive, versus being general to lots of applications while being too simple to predict the performance of any application very well.  Do you think the benchmark you chose does a good job with this balance?

- b. What assumptions does your benchmark make about the kind of machine that it is run on?  Do you think that those assumptions are reasonable?  Let's make this question very concrete: let's say you have access to [TaihuLight](https://en.wikipedia.org/wiki/Sunway_TaihuLight), whose nodes are neither really CPUs or GPUs, but somewhere in between.  Could your benchmark run on this machine?  If not, propose a way that you could change the benchmark to make it more portable.

- c. How exactly does your benchmark specify the way the problem is solved?  If your benchmark is for a particular algorithm or a particular code, do you think that the results of the benchmark would help you predict the performance of a different code/algorithm solving the same problem on the same machine?

- d. One measure of the complexity of a benchmark is how difficult it would be to write a reference implementation from scratch (one that solves the problem, if not in a "high-performance" way).  If you had to guess, how big would a team have to be do that: (i) one dedicated programmer; (ii) a team of about a dozen (like a research lab); (iii) an Organization (like a division of a company or a government agency)?  Give your reasoning (by, e.g. measuring lines of code in the implementation you will be working with)

**1.3 (1 pts):** Try to prepare for some of the logistics ahead of you.  Answer the following questions:

- a. Where / how will you obtain the source for the benchmark driver and implementation that you will be using? (Regarding how: is it a tarball, repository, or other?)

- b. What software environments will you need to build and run the benchmark? (e.g. Does it use raw `make`? Autotools?  CMake?  Is it python/pip/conda?  Does it need MPI?  OpenMP?  Cuda?)

**1.4 (1 pts):** Successfully install and run your benchmark

Include in this directory an example **job submission script** that runs your benchmark code.

In [1]:
module load cse6230/core
module load mkl/19.0

|                                                                         |
|       A note about python/3.6:                                          |
|       PACE is lacking the staff to install all of the python 3          |
|       modules, but we do maintain an anaconda distribution for          |
|       both python 2 and python 3. As conda significantly reduces        |
|       the overhead with package management, we would much prefer        |
|       to maintain python 3 through anaconda.                            |
|                                                                         |
|       All pace installed modules are visible via the module avail       |
|       command.                                                          |
|                                                                         |


In [2]:
lscpu | grep -E '^Thread|^Core|^Socket|^CPU\('
nvidia-smi
CPU_NAME=`cat /proc/cpuinfo| grep 'model name'|uniq | grep -P -m 1 -o -e "(?<=model name\s: ).*"`
CORE_COUNT=`cat /proc/cpuinfo |grep 'model name'|wc -l`
GPU_NAME=`nvidia-smi -q|grep 'Product Name'|uniq|grep -o "Tesla.*"`
GPU_COUNT=`nvidia-smi -q|grep 'Product Name'|wc -l`
echo "This nodes has ${CORE_COUNT} cores: its architecture is (Manufacturer, Product Id) ${CPU_NAME}"
if [[ ! $GPU_COUNT || $GPU_COUNT == 0 ]] ;  then
    echo "This node has no GPUs"
else
    echo "This node has ${GPU_COUNT} GPUs: its/their architecture is (Manufacturer, Product Id) ${GPU_NAME}"
fi

free -h

CPU(s):                28
Thread(s) per core:    1
Core(s) per socket:    14
Socket(s):             2
bash: nvidia-smi: command not found
This nodes has 28 cores: its architecture is (Manufacturer, Product Id) Intel(R) Xeon(R) CPU E5-2680 v4 @ 2.40GHz
This node has no GPUs
             total       used       free     shared    buffers     cached
Mem:          504G        40G       463G        38M       468M        33G
-/+ buffers/cache:       6.4G       498G 
Swap:         2.0G         0B       2.0G 


According to HPL suggestion, a good N should makes the program use 90% of total memory, which is $sqrt(130844220*1024/8)*0.9=116,472$. However, it takes too long (>6h) to compute. So we selected a N=11647, which is ten times smaller than that. For Block size, we also followed the recommendation to choose a value in the [32 , 256] interval. In our example, we selected 32, 64, 128, 256. Because the example test was performed on one node (chip), each core can be considered closely connected. Hence, 2 by 4 grid was set. **Example job submission script can be found in `cse6230-hw/final/script/pl_example.sh`**.

In [9]:
# testrun 
cd /nv/coc-ice/zjiang333/cse6230-hw/final/hpl/hpl-2.3/bin/Linux/
mpirun -np 28 ./xhpl 

HPLinpack 2.3  --  High-Performance Linpack benchmark  --   December 2, 2018
Written by A. Petitet and R. Clint Whaley,  Innovative Computing Laboratory, UTK
Modified by Piotr Luszczek, Innovative Computing Laboratory, UTK
Modified by Julien Langou, University of Colorado Denver

An explanation of the input/output parameters follows:
T/V    : Wall time / encoded variant.
N      : The order of the coefficient matrix A.
NB     : The partitioning blocking factor.
P      : The number of process rows.
Q      : The number of process columns.
Time   : Time in seconds to solve the linear system.
Gflops : Rate of execution for solving the linear system.

The following parameter values will be used:

N      :   19200    38400 
NB     :      64      192 
PMAP   : Row-major process mapping
P      :       4 
Q      :       7 
PFACT  :   Right 
NBMIN  :       4 
NDIV   :       2 
RFACT  :   Crout 
BCAST  :  1ringM 
DEPTH  :       0 
SWAP   : Mix (threshold = 64)
L1     : transposed form
U      : tra

**1.5 (3 pts):** Develop a performance model for your benchmark

In 1.1, you chose a performance metric of your benchmark, let's call it $V$.  Your benchmark will solve a problem with some parameters (problem size, the choice of matrix / network / etc.), let's call those parameters $N$.  The node that you chose to run on will have some machine parameters (The number of cores, the type of GPU, the bandwidth from main memory, etc., etc.), let's call them $P$.  Give an expression 
for $V(N,P)$ for your benchmark, and describe how you arrived at it.  You should use your discretion when choosing the level of detail.  If it is hard to develop a closed-form performance model for the whole benchmark, but there are a few key kernels that happens repeatedly in your benchmark (a stencil application, an iteration of stochastic gradient descent, etc.), you can give performance models for those benchmarks(s) instead.

If it is difficult to formulate your expression in terms of machine parameters, try to develop an expression
with coefficients that measure the rates at which the machine can do some lower-level operations (for example, the rate at which a GPU can sum an array).  If you have these coefficients, you should give a plausible description for how the architecture of the machine affects those rates.

**1.6 (2 pts):** Gather statistics for the performance metric

Include in this directory the **job script(s)** that you use to gather statistics for the performance metric on pace-ice.  Additionally, describe what steps you've taken to ensure the quality of the statistics: how are you accounting for variability / noise?  Does your benchmark show different performance on the first run than on subsequent runs?

If you are running your benchmark for multiple problem instances ($N$), include a plot of the performance metric for the different problem instances. (You can include error bars for maximum/minimum values of the performance metric for the same problem instance to convey variability.)

The script used can found in `cse6230-hw/final/script/hpl_fullscale.sh`. To ensure the quality of statistics, we have run the benchmark at the same setting for five times. 

In [4]:
cd /nv/coc-ice/zjiang333/cse6230-hw/final/hpl/hpl-2.3/bin/Linux/
mpirun -np 112 ./xhpl

HPLinpack 2.3  --  High-Performance Linpack benchmark  --   December 2, 2018
Written by A. Petitet and R. Clint Whaley,  Innovative Computing Laboratory, UTK
Modified by Piotr Luszczek, Innovative Computing Laboratory, UTK
Modified by Julien Langou, University of Colorado Denver

An explanation of the input/output parameters follows:
T/V    : Wall time / encoded variant.
N      : The order of the coefficient matrix A.
NB     : The partitioning blocking factor.
P      : The number of process rows.
Q      : The number of process columns.
Time   : Time in seconds to solve the linear system.
Gflops : Rate of execution for solving the linear system.

The following parameter values will be used:

N      :   19200    38400    57600    76800    96000   115200 
NB     :     192 
PMAP   : Row-major process mapping
P      :       8 
Q      :      14 
PFACT  :   Right 
NBMIN  :       4 
NDIV   :       2 
RFACT  :   Crout 
BCAST  :  1ringM 
DEPTH  :       0 
SWAP   : Mix (threshold = 64)
L1     : t

In [ ]:
#plot

**1.7 (3 pts):** Compare your performance model to your statistics

If your performance model allowed you to make a concrete prediction of $V(N,P)$ before running the benchmark,
compare (in a table or plot) the predictions and the actual measurements.

If your performance model includes coefficients that could not be estimated ahead of time, use measurements gathered during your experiments to get empirical values of those coefficients to fit the model to the data.  Once you do this, answer the following question: is there a plausible explanation (in terms of the architecture of the machine and the nature of the algorithm) for why these coefficients have the value that they do?

Present additional timings and/or machine performance metrics (either for the full benchmark or key kernels) and make a case for why you think they best demonstrate what the biggest bottleneck to the performance of the benchmark is.

## 2. Paper report (5 pts)

**Completed separately, not as a team**

Choose one of the two Gordon Bell Prize finalists this year:

- [A data-centric approach to extreme-scale ab initio dissipative quantum transport simulations](https://dl.acm.org/citation.cfm?id=3357156)
- [Fast, scalable and accurate finite-element based ab initio calculations using mixed precision computing: 46 PFLOPS simulation of a metallic dislocation system](https://dl.acm.org/citation.cfm?id=3357157)

Read the paper, and answer the following questions:

Answers to the first paper are as following:  

**a. What is the problem being solved?**   
  
A: It improves the computational efficiency of ab initio quantum transport (QT) solver. Large-scale QT simulations are bound by both communication volume and memory requirements. The former inhibits strong scaling, as simulation time includes nanostructure-dependent point-to-point communication patterns, which become infeasible when increasing node count. The memory bottleneck is a direct result of the former.  

**b.What are the most important kernels of the algorithm for solving the problem? (In this context consider a kernel to be a subproblem that is relevant to more than just the specific problem from a.)**   
   
A: They showed that the key to eliminating the scaling bottleneck is in formulating a communication-avoiding algorithm, which is tightly coupled with recovering local and global data dependencies of the application.  

The most important kernel is solving scattering self-energies (SSE) equations $\Sigma (E,k_z)$ and $\Pi (\omega, q_z)$. 95% of simulation time is dedicated to SSE, regardless of the number of used cores/nodes, among which ∼60% for the communication between the different MPI tasks.  

The method they used in the paper was a Data-Centric(DaCe) SSE, which reduces magnitudes of data movement and communications between ranks.  
**c. What about the important kernels and/or the size of the problem make this a challenging problem?**   
  
A: SSE are required to be evaluated for all $\Sigma (E,k_z)$ and $\Pi (\omega, q_z)$, so the computation cost increases with the number of atoms simulated. However, understanding realistic FinFET transistors requires simulations with $N_a ≥ 10, 000$ atoms and high accuracy. The SSE phase comsumption in SOTA result (OMEN) before this DaCe approch grows with the cores used, which could be attributed to communication overhead. To simulate the disired number of atoms with required accuracy, two orders of magnitude improvement in computation time per atom has to be made.    
**d. Summarize the innovation of this paper.**   
  
A: The innovations reported includes: 
1. Material was staged and chunked broadcasted to nodes, so data ingestion was ~15 times faster.  
2. Domain decomposation of SSE computation was tiled by atom position on the nano device instead of being controled by the top map as in OMEN. This modification reduces the movement needed and the complexity in MPI communications. 
3. Many kinds of computation were pipelined on GPU. For instance, contour integral calculation for boundary condition was pipelined. Copy/compute and compute/compute overlap were automatically generated by DaCe framework. 
4. In Green's Function (GF) computation, they investigated different DaCe transformation on sparse Hamiltonian blocks using a combination of sparse and dense matrices. 
5. Dataflow in SSE kernel was reformulated to provide speedup. For example, using map fission and data layout transformation, they reshaped the job into a stencil-like strided-batched dense multiplication, which yeiled ~5x speedup over cuBLAS.   
   
**e. What model is used that combines the problem parameters and machine parameters to predict performance?**  
  
A: For local computation, per-component benchmarks like matrix multiplication were performed on single node or GPU to provide actual local computation cost on certain system, which then help predict the overall perfrormance. For communication, by dividing the amount of data each node must sned by the injection bandwidth of that node, they preidcted a lower bounds for the completion time of each call. Overall performance wasn't predicted in the paper. Instead, actual performance is reported, and efficiency was calculated by dividing the achieved flop/s by peak flop/s measured. 

**f. Any paper that is submitted for a prize contains some marketing, and maybe some attempts to [fool the masses](https://blogs.fau.de/hager/archives/category/fooling-the-masses).  If you could ask the authors to submit one additional figure with the performance measurements of an experiment, what would you choose, and why?**  
  
A: I would ask the authors to add a figure about how the performance of cuPLAS and DaCe (SBSMM) change with the size of individual matrices used in the strided matrix multiplication. Because they claimed that cuBLAS excessively padded small matrices, hence the useful flops is low, although the seemingly high peak percentage. If the figure does support their hypothesis, we should expect something like the performance of cuBLAS increases with the size of matrices included while the DaCe doesn't. 